# --- Day 7: No Space Left On Device ---
https://adventofcode.com/2022/day/7

You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

$ system-update --please --pretty-please-with-sugar-on-top

Error: No space left on device

Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input).

The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:

cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
- cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
- cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
- cd / switches the current directory to the outermost directory, /.
- ls means list. It prints out all of the files and directories immediately contained by the current directory:
- 123 abc means that the current directory contains a file named abc with size 123.
- dir xyz means that the current directory contains a directory named xyz.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes.

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [2]:
def getTerminal():
    with open('termOut.txt') as file:
        return file.read()
print(getTerminal())

$ cd /
$ ls
dir blgtdv
dir dbrfcz
dir fvspj
dir hbjmndt
dir hzg
dir jpjgdm
dir mtd
dir pcpf
dir qfttswwv
dir qrd
dir zwqgg
$ cd blgtdv
$ ls
dir bjlcfcfq
83465 bnm
165543 cllbb
137127 cvbnjz.wzp
dir gzzz
dir nbdzs
dir vvmcfptr
dir wjvzwssp
$ cd bjlcfcfq
$ ls
dir fvspj
dir pwcvj
dir tvhrzql
$ cd fvspj
$ ls
315202 cqwb.qzt
12768 lmdtnsmr
$ cd ..
$ cd pwcvj
$ ls
dir bjp
138548 btj.sdn
140056 cllbb
dir drsgv
311875 fvspj
dir lpv
dir mmtsh
dir pwcvj
252680 qrd.ngg
dir rdftpm
dir rzhq
$ cd bjp
$ ls
dir scggvb
$ cd scggvb
$ ls
29012 lmdtnsmr
$ cd ..
$ cd ..
$ cd drsgv
$ ls
41717 btj.sdn
203587 jctnszj.nnc
304614 rdvjhjjr.bbt
$ cd ..
$ cd lpv
$ ls
292112 nlrbdjhf
304873 rdvjhjjr.bbt
$ cd ..
$ cd mmtsh
$ ls
12318 fbvvrvc.shb
$ cd ..
$ cd pwcvj
$ ls
dir dnfnl
$ cd dnfnl
$ ls
191701 qfswvhj.wgs
$ cd ..
$ cd ..
$ cd rdftpm
$ ls
157406 fvwcjbsl.tdr
59827 jfdbcwln.frw
$ cd ..
$ cd rzhq
$ ls
dir wcnm
$ cd wcnm
$ ls
149296 wljbd
$ cd ..
$ cd ..
$ cd ..
$ cd tvhrzql
$ ls
322339 hhlh.tdw
$ cd ..
$ cd ..


In [3]:
term=getTerminal().split('\n')

class Node:
    def __init__(self, name):
        self.data=name
        self.size=0
        self.children=[]
        self.parent=None
        
    def addChild(self, child):
        """Adds a directory as a child of current directory if it's not already there and sets parent = self"""
        child=Node(child)
        if child not in self.children:
            child.parent=self
            self.children.append(child)
            
    def printTree(self, ind=0):
        """Pretty prints the tree using ind as the number of indentations"""
        if ind:
            print("-"*(ind*2),str(self.data), self.size)
        else:
            print(self.data, self.size)
        if self.children:
            for child in self.children:
                child.printTree(ind+1)
                
    def postOrder(self):
        """Prints a post order traversal of the tree"""
        for i in self.children:
            i.postOrder()
        print(self.data)
        
    def postAddDirs(self):
        """Adds the size of all directories in self to self.size using a post order traversal"""
        for i in self.children:
            i.postAddDirs()
        if self.parent:
            self.parent.size+=self.size
            
    def getDirsSize(self, allDirs=[]):
        """Returns a list of all directory sizes that are <= 100000"""
        for i in self.children:
             allDirs=i.getDirsSize(allDirs)
        if self.size <=100000:
            allDirs.append(self.size)
        return allDirs
    
            
rootDir=Node(term[0][5]) #Sets root node to first line (/)
current=rootDir
for i in range(1,len(term)): #Skips the first line becasue it has already been set as the rootnode
    if term[i]=='$ ls':#If it shows all files in the directory, this will loop through all files
        x=i+1#Counter starting after $ ls
        while x!=len(term) and term[x][0]!='$':#Goes through all files in the directory
            if term[x][:3]=='dir': #If term[x] is a directory add as a child to current
                current.addChild(term[x][4:])
            elif term[x][0].isdigit(): #If term[x] is file, add size of file to current.size
                current.size+=(int(term[x][:term[x].index(' ')]))
            x+=1
    if term[i]!='$ cd ..' and term[i][:4]=='$ cd': #When going into a new cd
        for child in current.children: #Loops through all children
            if child.data==term[i][5:]: #If child.data == directory
                current=child #Set current = child
    if term[i]=='$ cd ..': #Sets current = parent
        current=current.parent
rootDir.postAddDirs()
#rootDir.printTree()

print(f'Sum of all directories with a total size of at most 100000: {sum(rootDir.getDirsSize())}')

Sum of all directories with a total size of at most 100000: 1667443


# --- Part Two ---
https://adventofcode.com/2022/day/7#part2

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

term="""$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""".split('\n')

In [4]:
term=getTerminal().split('\n')

class Node:
    def __init__(self, name):
        self.data=name #Name of the directory
        self.size=0 #Size of the directory
        self.children=[] 
        self.parent=None
        
    def addChild(self, child):
        """Adds a directory as a child of current directory if it's not already there and sets parent = self"""
        child=Node(child)
        if child not in self.children:
            child.parent=self
            self.children.append(child)
            
    def printTree(self, ind=0):
        """Pretty prints the tree using ind as the number of indentations"""
        if ind:#If there's any indentation
            print("-"*(ind*2),str(self.data), self.size)
        else:
            print(self.data, self.size)
        if self.children:#Recurse on all children increasing the indentation by 1 to represent a new level
            for child in self.children:
                child.printTree(ind+1)
                
    def postOrder(self):
        """Prints a post order traversal of the tree"""
        for i in self.children:
            i.postOrder()
        print(self.data)
        
    def postAddDirs(self):
        """Adds the size of all directories in self to self.size using a post order traversal"""
        for i in self.children:
            i.postAddDirs()
        if self.parent:
            self.parent.size+=self.size
            
    def getDirsSize(self, allDirs=[]):
        """Returns a list of all sizes of directories"""
        for i in self.children:
             allDirs=i.getDirsSize(allDirs)
        allDirs.append(self.size)
        return allDirs
            
rootDir=Node(term[0][5]) #Sets root node to first line (/)
current=rootDir
for i in range(1,len(term)): #Skips the first line becasue it has already been set as the rootnode
    if term[i]=='$ ls':#If it shows all files in the directory, this will loop through all files
        x=i+1#Counter starting after $ ls
        while x!=len(term) and term[x][0]!='$':#Goes through all files in the directory
            if term[x][:3]=='dir': #If term[x] is a directory add as a child to current
                current.addChild(term[x][4:])
            elif term[x][0].isdigit(): #If term[x] is file, add size of file to current.size
                current.size+=(int(term[x][:term[x].index(' ')]))
            x+=1
    if term[i]!='$ cd ..' and term[i][:4]=='$ cd': #When going into a new cd
        for child in current.children: #Loops through all children
            if child.data==term[i][5:]: #If child.data == directory
                current=child #Set current = child
    if term[i]=='$ cd ..': #Sets current = parent
        current=current.parent
rootDir.postAddDirs()
#rootDir.printTree()

spaceNeeded=30000000-(70000000-rootDir.size) #Gets the space needed to clear
smallestDir=min([x for x in rootDir.getDirsSize() if x >= spaceNeeded]) #Gets the min of the list of numbers <=spaceNeeded
print(f'Size of smallest directory big enough to delete: {smallestDir}')

Size of smallest directory big enough to delete: 8998590


# Printing Terminal Directories as a Tree
(This is just here because I like it and think it looks cool)

In [5]:
term=getTerminal().split('\n')

class Node:
    def __init__(self, name):
        self.data=name #Name of the directory
        self.size=0 #Size of the directory
        self.children=[] 
        self.parent=None
        
    def addChild(self, child):
        """Adds a directory as a child of current directory if it's not already there and sets parent = self"""
        child=Node(child)
        if child not in self.children:
            child.parent=self
            self.children.append(child)
            
    def printTree(self, ind=0, printNums=False):
        """Pretty prints the tree using ind as the number of indentations
        printNums takes in a boolean value that decides to print or not print numbers after each directory"""
        if printNums:
            if ind:#If there's any indentation
                print("-"*(ind*2),str(self.data), self.size)
            else:
                print(self.data, self.size)
            if self.children:#Recurse on all children increasing the indentation by 1 to represent a new level
                for child in self.children:
                    child.printTree(ind+1, printNums)
        else:
            if ind:#If there's any indentation
                print("-"*(ind*2),str(self.data))
            else:
                print(self.data)
            if self.children:#Recurse on all children increasing the indentation by 1 to represent a new level
                for child in self.children:
                    child.printTree(ind+1, printNums)
    
            
rootDir=Node(term[0][5]) #Sets root node to first line (/)
current=rootDir
for i in range(1,len(term)): #Skips the first line becasue it has already been set as the rootnode
    if term[i]=='$ ls':#If it shows all files in the directory, this will loop through all files
        x=i+1#Counter starting after $ ls
        while x!=len(term) and term[x][0]!='$':#Goes through all files in the directory
            if term[x][:3]=='dir': #If term[x] is a directory add as a child to current
                current.addChild(term[x][4:])
            elif term[x][0].isdigit(): #If term[x] is file, add size of file to current.size
                current.size+=(int(term[x][:term[x].index(' ')]))
            x+=1
    if term[i]!='$ cd ..' and term[i][:4]=='$ cd': #When going into a new cd
        for child in current.children: #Loops through all children
            if child.data==term[i][5:]: #If child.data == directory
                current=child #Set current = child
    if term[i]=='$ cd ..': #Sets current = parent
        current=current.parent

rootDir.printTree(printNums=False)

/
-- blgtdv
---- bjlcfcfq
------ fvspj
------ pwcvj
-------- bjp
---------- scggvb
-------- drsgv
-------- lpv
-------- mmtsh
-------- pwcvj
---------- dnfnl
-------- rdftpm
-------- rzhq
---------- wcnm
------ tvhrzql
---- gzzz
---- nbdzs
---- vvmcfptr
---- wjvzwssp
-- dbrfcz
-- fvspj
---- cdc
---- fvspj
------ pwcvj
------ qrd
---- pwcvj
---- qtnnz
-- hbjmndt
-- hzg
-- jpjgdm
---- fpgcdvm
------ lsj
-------- hdttgfvj
---------- pwcvj
------ qrd
-------- btltml
-------- rggvfrgb
---- fqh
------ fvspj
-------- fvspj
------ lpv
------ pdqss
---- mcszw
------ hdcpljg
-------- cwrzb
------ jpgrltj
-------- frqzwfbd
---------- rbpwmbg
-------- fvspj
---------- bjlcfcfq
-------- mhrnnhhz
---------- qrd
---------- vwtrwbc
-------- mmf
---------- dvtbrnp
---------- qrd
---------- rfnnt
---------- sgrnztrg
------------ fqzt
-------------- jrvwtcm
---------------- lpv
------------------ jnzjz
---------------- qrd
-------------- lvwvhh
-------------- qdmwttjf
------------ fwnfzlsh
------------ n